In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import RadiusNeighborsClassifier

remove_fight_island = False    #Removes fight island fights from consideration

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)





4804

In [6]:
#The fight island fights were messing with the established models.  This function most likely isn't needed anymore
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_no_fight_island = df[(df['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    #df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df = df_no_fight_island
    #df_train = df_train_no_fight_island

len(df)

4804

In [7]:
num_upcoming_fights = len(df_upcoming)

In [8]:
df = df_upcoming.append(df)
len(df)

4817

In [9]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-e6abcb5e0e70>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-9-e6abcb5e0e70>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [10]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [11]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [12]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

13
13
13
4804
4804
4804


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Cory Sandhagen,TJ Dillashaw,-210,175,47.619048,175.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Blue,...,5.0,5:00,1500.0,500.0,500.0,600.0,1100.0,110.0,400.0,1.0
1,Kyler Phillips,Raulian Paiva,-320,250,31.250000,250.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,5:00,900.0,-125.0,500.0,1000.0,1000.0,400.0,900.0,1.0
2,Darren Elkins,Darrick Minner,130,-155,130.000000,64.516129,2021-07-24,"Las Vegas, Nevada, USA",USA,Red,...,2.0,3:48,528.0,275.0,250.0,600.0,240.0,800.0,600.0,0.0
3,Miranda Maverick,Maycee Barber,-155,135,64.516129,135.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,5:00,900.0,120.0,275.0,650.0,1400.0,750.0,450.0,1.0
4,Adrian Yanez,Randy Costa,-240,195,41.666667,195.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,Red,...,2.0,2:11,431.0,380.0,500.0,1100.0,1600.0,-110.0,450.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,1.0,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4800,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4801,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4802,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Uriah Hall,Sean Strickland,180,-220,180.000000,45.454545,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,650.0,200.0,2500.0,800.0,330.0,175.0,NaN
1,Kyung Ho Kang,Rani Yahya,-130,110,76.923077,110.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,140.0,450.0,750.0,240.0,600.0,1400.0,NaN
2,Cheyanne Buys,Gloria de Paula,-170,150,58.823529,150.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,-115.0,240.0,1000.0,1400.0,800.0,900.0,NaN
3,Niklas Stolze,Jared Gooden,-200,170,50.000000,170.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,115.0,405.0,NaN,NaN,NaN,NaN,NaN
4,Ryan Benoit,Zarrukh Adashev,-135,115,74.074074,115.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,250.0,NaN,1800.0,NaN,400.0,NaN
5,Bryan Barberena,Jason Witt,-275,220,36.363636,220.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,250.0,550.0,550.0,750.0,130.0,850.0,NaN
6,Nicco Montano,Wu Yanan,-250,200,40.000000,200.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,-135.0,350.0,600.0,1400.0,800.0,1000.0,NaN
7,Collin Anglin,Melsik Baghdasaryan,115,-135,115.000000,74.074074,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,240.0,225.0,900.0,1000.0,550.0,250.0,NaN
8,Chris Gruetzemacher,Rafa Garcia,250,-320,250.000000,31.250000,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,450.0,150.0,1600.0,200.0,900.0,400.0,NaN
9,Danny Chavez,Kai Kamaka,-105,-115,95.238095,86.956522,2021-07-31,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,240.0,NaN,1600.0,NaN,330.0,NaN,NaN


In [15]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [16]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    #print(df_train)
    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    

    return(ev_prepped_df)

In [17]:
with open('../data/production_models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
1
2
3
4
5
6
7
8
9
YES
10
11
12
The model choice is 9
[['model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20', 'model_10_23_20', 'model_10_27_20', 'model_10_28_20', 'model_2021_06_24', 'model_2021_06_30', 'model_2021_07_21'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoothing=1e-12)', 'GaussianNB(var_smoothing=1e-12)', 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)", "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(max_depth=95, min_samples_leaf

In [18]:

output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Uriah Hall vs. Sean Strickland
Uriah Hall has an EV of 63.86 on odds of 180.  They have 58.52 % chance of winning.

Kyung Ho Kang vs. Rani Yahya
Rani Yahya has an EV of 15.03 on odds of 110.  They have 54.78 % chance of winning.

Cheyanne Buys vs. Gloria de Paula
No bets.  EV1:4.123604216333447. EV2: -13.898265896080424
Cheyanne Buys has an EV of 4.12 on odds of -170.  They have 65.56 % chance of winning.

Niklas Stolze vs. Jared Gooden
No bets.  EV1:3.974512821372464. EV2: -17.154123078470427
Niklas Stolze has an EV of 3.97 on odds of -200.  They have 69.32 % chance of winning.

Ryan Benoit vs. Zarrukh Adashev
No bets.  EV1:3.7739756541930305. EV2: -13.171899717572465
Ryan Benoit has an EV of 3.77 on odds of -135.  They have 59.61 % chance of winning.

Bryan Barberena vs. Jason Witt
No bets.  EV1:-2.4349366761365907. EV2: -8.952681933332762
Bryan Barberena has an EV of -2.43 on odds of -275.  They have 71.55 % chance of winning.

Nicco Montano vs. Wu Yanan
Nicco Montano has an EV of 6

In [19]:
output_df

,t1_name,t2_name,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,Uriah Hall,Sean Strickland,180,-220,0.585230,0.414770,NaN
1,Kyung Ho Kang,Rani Yahya,-130,110,0.452230,0.547770,NaN
2,Cheyanne Buys,Gloria de Paula,-170,150,0.655593,0.344407,NaN
3,Niklas Stolze,Jared Gooden,-200,170,0.693163,0.306837,NaN
4,Ryan Benoit,Zarrukh Adashev,-135,115,0.596148,0.403852,NaN
5,Bryan Barberena,Jason Witt,-275,220,0.715477,0.284523,NaN
6,Nicco Montano,Wu Yanan,-250,200,0.761878,0.238122,NaN
7,Collin Anglin,Melsik Baghdasaryan,115,-135,0.550584,0.449416,NaN
8,Chris Gruetzemacher,Rafa Garcia,250,-320,0.504542,0.495458,NaN
9,Danny Chavez,Kai Kamaka,-105,-115,0.598388,0.401612,NaN


In [20]:
return_df = output_df[['t1_name', 't2_name', 't1_prob', 't2_prob']]

In [21]:
return_df

,t1_name,t2_name,t1_prob,t2_prob
0,Uriah Hall,Sean Strickland,0.585230,0.414770
1,Kyung Ho Kang,Rani Yahya,0.452230,0.547770
2,Cheyanne Buys,Gloria de Paula,0.655593,0.344407
3,Niklas Stolze,Jared Gooden,0.693163,0.306837
4,Ryan Benoit,Zarrukh Adashev,0.596148,0.403852
5,Bryan Barberena,Jason Witt,0.715477,0.284523
6,Nicco Montano,Wu Yanan,0.761878,0.238122
7,Collin Anglin,Melsik Baghdasaryan,0.550584,0.449416
8,Chris Gruetzemacher,Rafa Garcia,0.504542,0.495458
9,Danny Chavez,Kai Kamaka,0.598388,0.401612


In [22]:
output_df = return_df.rename(columns = {'t1_name': 'R_fighter', 't2_name': 'B_fighter', 't1_prob': 'R_prob', 't2_prob': 'B_prob'})

In [23]:
output_df.to_csv('../data/upcoming_probs.csv', index=False)